# Voting

In [18]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

f_size = 16
# Importing the dataset
dataset = pd.read_csv('CVD dataset2.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, f_size].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
17


In [19]:
dataNoCvd = data[~(data[:,f_size] > 0.0)]
dataCvd = data[~(data[:,f_size] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [20]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

In [21]:
def create_train_test_sets(cv,dataNoCvd,dataCvd):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)

    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)

        testSubset_NoCvd ,testSubset_Cvd ,testSubset_total = find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
     
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, f_size, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, f_size), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
    
        X_train.append(np.delete(trainSubset_total, f_size, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, f_size), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, train_total_NoCvd, train_total_Cvd

In [22]:
train_total, test_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(10,dataNoCvd,dataCvd)

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9


In [23]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)

504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07142857142857142
ratio in test set: 0.08928571428571429


In [31]:
import numpy as np
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from statistics import mean


pbounds = {
#     'learning_rate': (0.01, 1.0),
#     'n_estimators': (100, 1000),
#     'min_child_weight': (2,5),
#     'max_depth': (2,8),
#     'subsample': (0.6, 1.0),  # Change for big datasets
#     'colsample_bylevel': (0.5, 1.0),  # Change for datasets with lots of features
#     'reg_lambda': (0.0,3.0),
#     'colsample_bytree':(0.5,1.0),
#     'scale_pos_weight':(1,7),
#     'reg_alpha':(0,0.3),
#     'gamma': (0, 5)}

     'learning_rate': (0.01, 1.0),
    'n_estimators': (100, 1000),
    'min_child_weight': (1,5),
    'max_depth': (2,5),
    'subsample': (0.8, 1.0),  # Change for big datasets
    'colsample_bylevel': (0.5, 1.0),  # Change for datasets with lots of features
    'reg_lambda': (1.0,2.0),
    'colsample_bytree':(0.5,1.0),
    'scale_pos_weight':(1,6),
    'reg_alpha':(0,0.2),
    'gamma': (0, 5)}

def xgboost_hyper_param(learning_rate,
                        n_estimators,
                        min_child_weight,
                        max_depth,
                        subsample,
                        colsample_bylevel,
                        colsample_bytree,
                        reg_lambda,
                        scale_pos_weight,
                        reg_alpha,
                        gamma):

    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    min_child_weight = int(min_child_weight)
    

  

    clf = XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_child_weight = min_child_weight,
        n_estimators=n_estimators,
        subsample=subsample,
        colsample_bylevel=colsample_bylevel,
        reg_lambda=reg_lambda,
        colsample_bytree=colsample_bytree,
        scale_pos_weight=scale_pos_weight,
        reg_alpha=reg_alpha,
        gamma=gamma)
        
    accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,clf)

    print("accuraccy:    ", mean(accuracy))
    print("specificity:  ", mean(specificity))
    print("sensitivity:  ", mean(sensitivity))
#     return np.mean([roc_auc_score(X, y),roc_auc_score(X, y)])
#     return np.mean(cross_val_score(clf, X, y, cv=3, scoring='roc_auc'))
    return np.mean(auc)
    




In [14]:
print(limits)

NameError: name 'limits' is not defined

In [25]:
from xgboost import XGBClassifier
limits= [[],[],[],[],[],[]]

def my_fit(ratio,dataNoCvd,dataCvd,clf):   
    Subarray=[]
    models = []
    
    # yarray
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
#     print("Size of NoCvd cases in train set:",rowsNoCvd)
#     print("Size of Cvd cases in train set  :",rowsCvd)


    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    numOfSubsamples= int(numOfSubsamples)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples

#     print(SubNoCvd,residue)
    Up = 0


    for i in range(numOfSubsamples):
        
        
#         classifier = XGBClassifier()
    #     print(i)
    #Creating the training set for each model
        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
        limits[i].append([Down,Up])
        
#         print(Down)
#         print(Up)
        
        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], f_size, axis=1)
        y=np.delete(Subarray[i], slice(0, f_size), axis=1)
        y=np.reshape(y, len(y))
        
        clf.fit( X, y)
        models.append(clf)
    return models     

In [26]:
def voting(models,X):
    y_pred = []
    y_pred_models = []

    models_size = len(models)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict(X))
        
    for j in range(X_size):
        case = 0
        for i in range(models_size):
            if y_pred_models[i][j] == 1.0:
                case+=1
        if case >  models_size/2 :
            y_pred.append(1.) 
        else:
            y_pred.append(0.) 
            
    return y_pred  

In [27]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, train_total, test_total, train_total_NoCvd, train_total_Cvd,clf):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        


        models = my_fit( 2, train_total_NoCvd[i], train_total_Cvd[i],clf)


        y_pred = voting(models,X_test)

        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [30]:
optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)

optimizer.maximize(
    init_points=3,
    n_iter=10,
)
print(optimizer.max)



|   iter    |  target   | colsam... | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
accuraccy:     0.6589285714285714
specificity:   0.6688536953242835
sensitivity:   0.54
|  1        |  0.6044   |  0.8834   |  0.8602   |  0.000571 |  0.3093   |  2.44     |  1.369    |  267.6    |  0.06911  |  1.397    |  3.694    |  0.8838   |
accuraccy:     0.49642857142857144
specificity:   0.4896681749622926
sensitivity:   0.59
|  2        |  0.5398   |  0.937    |  0.6022   |  4.391    |  0.03711  |  4.011    |  2.669    |  602.8    |  0.02808  |  1.198    |  5.004    |  0.9937   |
accuraccy:     0.5392857142857143
specificity:   0.5337858220211161
sensitivity:   0.615
|  3        |  0.5744   |  0.8627   |  0.8462   |  4.382    |  0.8957   |  2.255    |  1.156    |  252

In [243]:
print(params)

[[0.0371137172659469, 7, 603, 3, 4.390587181954727, 0.8426097501983798, 1.1981014890848787, 5, 0.6022261248657588, 0.028077387719046755, 0.9936523151438795], [1.0, 2, 520, 5, 0.0, 0.9999997552748909, 1.0, 6, 0.9999999615037215, 0.15658803239216332, 1.0], [0.0371137172659469, 7, 603, 3, 4.390587181954727, 0.8426097501983798, 1.1981014890848787, 5, 0.6022261248657588, 0.028077387719046755, 0.9936523151438795], [0.0371137172659469, 7, 603, 3, 4.390587181954727, 0.8426097501983798, 1.1981014890848787, 5, 0.6022261248657588, 0.028077387719046755, 0.9936523151438795], [0.6145449281071235, 9, 100, 4, 0.21956496627989208, 0.8897165176410781, 1.8947636173075781, 6, 0.9292908434215305, 0.17662095712781303, 0.9599449263854488], [0.6145449281071235, 9, 100, 4, 0.21956496627989208, 0.8897165176410781, 1.8947636173075781, 6, 0.9292908434215305, 0.17662095712781303, 0.9599449263854488]]


In [170]:
optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=pbounds,
    random_state=1,
)

In [26]:
optimizer.maximize(
    init_points=2,
    n_iter=500,
)
print(optimizer.max)

|   iter    |  target   | colsam... | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
accuraccy:     0.5089285714285714
specificity:   0.5110105580693816
sensitivity:   0.495
|  103      |  0.503    |  0.6567   |  0.8462   |  4.382    |  0.8957   |  2.255    |  1.156    |  252.8    |  0.1756   |  1.098    |  3.106    |  0.9916   |
accuraccy:     0.6285714285714286
specificity:   0.635972850678733
sensitivity:   0.54
|  104      |  0.588    |  0.7666   |  0.8459   |  1.578    |  0.6896   |  4.504    |  1.073    |  775.1    |  0.1978   |  1.748    |  2.402    |  0.9579   |
accuraccy:     0.6714285714285714
specificity:   0.6863499245852187
sensitivity:   0.49
|  105      |  0.5882   |  0.5      |  1.0      |  0.0      |  1.0      |  5.0      |  1.0      |  757.8

accuraccy:     0.5857142857142857
specificity:   0.6053544494720965
sensitivity:   0.345
|  126      |  0.4752   |  1.0      |  0.5      |  0.0      |  1.0      |  5.0      |  5.0      |  950.4    |  0.2      |  2.0      |  1.0      |  0.8      |
accuraccy:     0.8321428571428572
specificity:   0.884577677224736
sensitivity:   0.17
|  127      |  0.5273   |  1.0      |  0.5      |  0.0      |  0.01     |  5.0      |  1.0      |  110.6    |  0.0      |  2.0      |  1.0      |  0.8      |
accuraccy:     0.6285714285714286
specificity:   0.6380090497737556
sensitivity:   0.515
|  128      |  0.5765   |  0.5      |  0.5      |  0.0      |  1.0      |  2.0      |  1.0      |  419.9    |  0.0      |  2.0      |  6.0      |  1.0      |
accuraccy:     0.6178571428571429
specificity:   0.6246606334841629
sensitivity:   0.535
|  129      |  0.5798   |  1.0      |  0.5      |  0.0      |  1.0      |  5.0      |  1.0      |  360.8    |  0.2      |  2.0      |  6.0      |  0.8      |
accuraccy:    

accuraccy:     0.6178571428571429
specificity:   0.6436651583710408
sensitivity:   0.3
|  150      |  0.4718   |  0.5      |  0.5      |  0.0      |  1.0      |  5.0      |  5.0      |  380.4    |  0.0      |  2.0      |  1.0      |  0.8      |
accuraccy:     0.1357142857142857
specificity:   0.07326546003016592
sensitivity:   0.93
|  151      |  0.5016   |  0.5      |  0.5      |  5.0      |  0.01     |  2.0      |  1.0      |  441.3    |  0.2      |  2.0      |  6.0      |  1.0      |
accuraccy:     0.7892857142857143
specificity:   0.834502262443439
sensitivity:   0.22
|  152      |  0.5273   |  0.5      |  0.5      |  0.0      |  0.01     |  2.0      |  1.0      |  321.0    |  0.2      |  2.0      |  1.0      |  1.0      |
accuraccy:     0.7
specificity:   0.7188914027149321
sensitivity:   0.46
|  153      |  0.5894   |  0.5      |  1.0      |  0.0      |  1.0      |  2.0      |  1.0      |  125.6    |  0.0      |  2.0      |  1.0      |  1.0      |
accuraccy:     0.480357142857142

accuraccy:     0.5464285714285715
specificity:   0.5397058823529411
sensitivity:   0.635
|  174      |  0.5874   |  0.5      |  1.0      |  5.0      |  1.0      |  2.0      |  1.0      |  812.6    |  0.2      |  1.0      |  6.0      |  0.8      |
accuraccy:     0.45535714285714285
specificity:   0.4530920060331825
sensitivity:   0.49
|  175      |  0.4715   |  1.0      |  0.5      |  5.0      |  1.0      |  2.0      |  5.0      |  704.5    |  0.0      |  1.0      |  6.0      |  0.8      |
accuraccy:     0.5339285714285714
specificity:   0.5301282051282051
sensitivity:   0.59
|  176      |  0.5601   |  0.5      |  1.0      |  5.0      |  1.0      |  2.0      |  1.0      |  360.8    |  0.0      |  1.0      |  6.0      |  0.8      |
accuraccy:     0.7232142857142857
specificity:   0.7478129713423831
sensitivity:   0.415
|  177      |  0.5814   |  1.0      |  1.0      |  0.0      |  0.01     |  5.0      |  1.0      |  910.1    |  0.0      |  2.0      |  1.0      |  1.0      |
accuraccy:   

accuraccy:     0.7482142857142857
specificity:   0.7806938159879336
sensitivity:   0.345
|  198      |  0.5628   |  0.5      |  1.0      |  0.0      |  0.01     |  2.0      |  3.309    |  228.0    |  0.2      |  2.0      |  1.0      |  0.8      |
accuraccy:     0.6964285714285714
specificity:   0.719079939668175
sensitivity:   0.415
|  199      |  0.567    |  0.7333   |  0.9538   |  1.166    |  0.1033   |  2.684    |  1.35     |  682.9    |  0.1389   |  1.632    |  1.141    |  0.9971   |
accuraccy:     0.7589285714285714
specificity:   0.8038084464555053
sensitivity:   0.2
|  200      |  0.5019   |  0.5      |  1.0      |  0.0      |  0.01     |  2.0      |  5.0      |  300.6    |  0.2      |  2.0      |  1.0      |  0.8      |
accuraccy:     0.8714285714285714
specificity:   0.9384615384615385
sensitivity:   0.025
|  201      |  0.4817   |  1.0      |  1.0      |  5.0      |  0.01     |  2.0      |  1.0      |  735.9    |  0.2      |  1.0      |  1.0      |  1.0      |
accuraccy:     

...

accuraccy:     0.45
specificity:   0.43367269984917045
sensitivity:   0.66
|  486      |  0.5468   |  0.5      |  1.0      |  5.0      |  1.0      |  2.0      |  1.0      |  343.6    |  0.0      |  2.0      |  6.0      |  1.0      |
accuraccy:     0.6321428571428571
specificity:   0.639894419306184
sensitivity:   0.535
|  487      |  0.5874   |  0.5      |  0.5      |  0.0      |  1.0      |  5.0      |  1.0      |  437.8    |  0.2      |  1.0      |  6.0      |  0.8      |
accuraccy:     0.4446428571428571
specificity:   0.42571644042232276
sensitivity:   0.685
|  488      |  0.5554   |  0.5      |  1.0      |  4.988    |  1.0      |  2.295    |  1.0      |  577.2    |  0.2      |  2.0      |  6.0      |  1.0      |
accuraccy:     0.5482142857142858
specificity:   0.5398567119155354
sensitivity:   0.665
|  489      |  0.6024   |  0.5      |  1.0      |  4.02     |  1.0      |  5.0      |  1.0      |  488.7    |  0.2      |  1.0      |  6.0      |  0.8      |
accuraccy:     0.203571428

accuraccy:     0.39285714285714285
specificity:   0.36425339366515835
sensitivity:   0.76
|  510      |  0.5621   |  0.5      |  1.0      |  5.0      |  0.01     |  5.0      |  1.0      |  494.8    |  0.2      |  2.0      |  6.0      |  0.8      |
accuraccy:     0.7196428571428571
specificity:   0.7440045248868778
sensitivity:   0.41
|  511      |  0.577    |  1.0      |  1.0      |  5.0      |  1.0      |  5.0      |  1.0      |  169.7    |  0.2      |  2.0      |  1.0      |  0.8      |
accuraccy:     0.6321428571428571
specificity:   0.6341628959276018
sensitivity:   0.61
|  512      |  0.6221   |  1.0      |  0.5      |  0.0      |  1.0      |  2.0      |  1.0      |  121.1    |  0.0      |  2.0      |  6.0      |  1.0      |
accuraccy:     0.6857142857142857
specificity:   0.6996229260935143
sensitivity:   0.51
|  513      |  0.6048   |  0.5      |  0.5      |  0.0      |  1.0      |  2.0      |  1.0      |  758.3    |  0.2      |  2.0      |  1.0      |  0.8      |
accuraccy:    

accuraccy:     0.6071428571428571
specificity:   0.6111613876319759
sensitivity:   0.565
|  534      |  0.5881   |  0.5      |  1.0      |  5.0      |  1.0      |  3.118    |  1.0      |  502.7    |  0.0      |  1.0      |  6.0      |  0.8      |
accuraccy:     0.5375
specificity:   0.5397812971342383
sensitivity:   0.51
|  535      |  0.5249   |  0.5389   |  0.7598   |  4.707    |  0.9042   |  4.297    |  1.404    |  268.5    |  0.03933  |  1.018    |  3.766    |  0.9262   |
accuraccy:     0.3892857142857143
specificity:   0.36621417797888384
sensitivity:   0.685
|  536      |  0.5256   |  1.0      |  0.5      |  0.0      |  0.01     |  2.0      |  2.239    |  864.2    |  0.0      |  2.0      |  6.0      |  0.8      |
accuraccy:     0.7
specificity:   0.7248114630467571
sensitivity:   0.39
|  537      |  0.5574   |  0.9812   |  0.5444   |  0.8889   |  0.5999   |  3.69     |  1.186    |  254.0    |  0.1874   |  1.911    |  1.025    |  0.887    |
accuraccy:     0.3625
specificity:   0.3

accuraccy:     0.5267857142857143
specificity:   0.5280920060331825
sensitivity:   0.51
|  558      |  0.519    |  1.0      |  1.0      |  0.0      |  1.0      |  2.366    |  4.453    |  832.1    |  0.2      |  2.0      |  5.929    |  0.8      |
accuraccy:     0.5678571428571428
specificity:   0.5686274509803921
sensitivity:   0.56
|  559      |  0.5643   |  1.0      |  0.5      |  0.0      |  0.01     |  2.0      |  1.0      |  398.3    |  0.0      |  2.0      |  2.719    |  0.8      |
accuraccy:     0.6589285714285714
specificity:   0.6688536953242835
sensitivity:   0.535
|  560      |  0.6019   |  0.5      |  1.0      |  0.0      |  0.01     |  3.542    |  1.0      |  956.1    |  0.0      |  2.0      |  2.118    |  1.0      |
accuraccy:     0.6678571428571428
specificity:   0.6804298642533937
sensitivity:   0.51
|  561      |  0.5952   |  1.0      |  1.0      |  0.0      |  1.0      |  2.802    |  2.104    |  344.1    |  0.2      |  1.0      |  1.0      |  1.0      |
accuraccy:     

accuraccy:     0.8142857142857143
specificity:   0.8710030165912519
sensitivity:   0.095
|  582      |  0.483    |  0.5      |  0.5      |  5.0      |  1.0      |  2.0      |  5.0      |  873.1    |  0.2      |  2.0      |  1.012    |  1.0      |
accuraccy:     0.5107142857142857
specificity:   0.4992081447963801
sensitivity:   0.66
|  583      |  0.5796   |  0.595    |  0.7781   |  4.59     |  0.8523   |  4.578    |  1.214    |  173.6    |  0.1935   |  1.397    |  5.906    |  0.8627   |
accuraccy:     0.5035714285714286
specificity:   0.49536199095022626
sensitivity:   0.615
|  584      |  0.5552   |  0.5561   |  0.5101   |  4.981    |  0.7663   |  4.243    |  1.357    |  337.2    |  0.126    |  1.732    |  5.91     |  0.9641   |
accuraccy:     0.6071428571428571
specificity:   0.6166666666666667
sensitivity:   0.485
|  585      |  0.5508   |  1.0      |  0.5      |  0.0      |  1.0      |  2.0      |  1.0      |  365.8    |  0.2      |  1.0      |  6.0      |  1.0      |
accuraccy:  

In [168]:
optimizer.maximize(
    init_points=2,
    n_iter=5,
)
print(optimizer.max)

|   iter    |  target   | colsam... | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... | scale_... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        |  0.6274   |  0.7085   |  0.8602   |  0.000571 |  0.3093   |  3.174    |  1.369    |  267.6    |  0.06911  |  1.397    |  3.694    |  0.8838   |
|  2        |  0.6486   |  0.8426   |  0.6022   |  4.391    |  0.03711  |  7.364    |  2.669    |  602.8    |  0.02808  |  1.198    |  5.004    |  0.9937   |
|  3        |  0.6456   |  0.7019   |  0.7023   |  1.073    |  0.0207   |  3.28     |  1.283    |  999.5    |  0.1539   |  1.004    |  2.399    |  0.8593   |
|  4        |  0.5607   |  0.7153   |  0.6043   |  0.8948   |  0.6301   |  2.096    |  4.599    |  752.6    |  0.08797  |  1.249    |  1.31     |  0.8599   |
|  5        |  0.6319   |  0.5793   |  0.9521   |  2

In [ ]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100,200,300,400],
                    "min_child_weight" :[1,2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3,4,5],
                    "colsample_bytree" :[0.5,0.75],
                    "reg_alpha"        :[0.05,0.1,0.2],
                    "subsample"        :[0.8]
    
        }

In [143]:
params = [[],[],[],[],[],[]]
for i in range(6):
    best_par = optimizer.max.get('params')
    params.append(best_par.get('learning_rate'))
    params.append(round(best_par.get('max_depth')))
    params.append(round(best_par.get('n_estimators')))
    params.append(round(best_par.get('min_child_weight')))
    params.append(best_par.get('gamma'))
    params.append(best_par.get('colsample_bylevel'))
    params.append(best_par.get('reg_lambda'))
    params.append(round(best_par.get('scale_pos_weight')))
    params.append(best_par.get('colsample_bytree'))
    params.append(best_par.get('reg_alpha'))
    params.append(best_par.get('subsample'))

print(o)

3.740878001587038


In [138]:
optimizer.maximize(
    init_points=2,
    n_iter=3,
)

|   iter    |  target   | colsample |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------
|  8        |  0.6205   |  1.0      |  3.352    |  0.4231   |  6.47     |  1.562    |  278.3    |  0.9601   |
|  9        |  0.5574   |  1.0      |  1.567    |  0.6954   |  9.011    |  4.578    |  176.5    |  0.8078   |
|  10       |  0.5911   |  1.0      |  5.0      |  0.01     |  2.0      |  1.0      |  100.0    |  0.8      |
|  11       |  0.5535   |  1.0      |  0.0      |  1.0      |  2.0      |  5.0      |  746.0    |  1.0      |
|  12       |  0.6151   |  1.0      |  0.6183   |  0.04311  |  2.119    |  4.503    |  578.5    |  0.9983   |
